In [ ]:
model_path = r'..\models\fl_fedavg\effnet-b0\ff_effnet0_1fc_fl.pth'
image_path = r'test_deepfake.png'
image_path = r'test_original.png'


import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
from efficientnet_pytorch import EfficientNet
from facenet_pytorch import MTCNN
from PIL import Image
import timm

device = 'cuda'

model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=1)
# model = timm.create_model('xception', pretrained=True, num_classes=1)
model.load_state_dict(torch.load(model_path)['model_state'])
model.to(device)
model.eval() 


In [ ]:
IMG_SIZE = 224


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

class ImageTransform:
    def __init__(self, size, mean, std, train=False):
        self.data_transform = transforms.Compose([
            transforms.Resize((size, size), interpolation=Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, img):
        return self.data_transform(img)



transformer = ImageTransform(IMG_SIZE, mean, std, train=False)

In [ ]:

img = Image.open(image_path).convert('RGB')
mtcnn = MTCNN(margin=300, select_largest=False, factor=0.5, device=device, post_process=False) # post_process=False if want human readable image

boxes, probs = mtcnn.detect(img)


if boxes is not None:
    print(f"Number of faces detected: {boxes.shape[0]}")

    for i, box in enumerate(boxes):
        print(f"Face {i + 1}: {box} {probs[i]}")

        face = img.crop((box[0], box[1], box[2], box[3]))
        
        face.show() 

        face_tensor = transformer(face).unsqueeze(0).to(device)  

        with torch.no_grad():
            output = torch.sigmoid(model(face_tensor))
            prob = output.item()  # get float value
            print(prob)
            predicted_class = int(prob > 0.5)  # higher prob means more likely original
            is_deepfake = 'Pristine' if predicted_class else 'Deepfake'
            print(f"Face {i+1}: Predicted class index = {predicted_class} {is_deepfake}")

else:
    print("No faces detected.")

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torc

Loaded pretrained weights for efficientnet-b0
Number of faces detected: 1
Face 1: [361.0030822753906 89.38159942626953 483.9775695800781 259.80963134765625] 0.9998471736907959


C:\Users\Sheryl\AppData\Local\Temp\ipykernel_14776\1760510307.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['model_state']

0.5730893015861511
Face 1: Predicted class index = 1 Pristine
